# Keras Tuner
Keras tuner is used to try different models based on different number of neurons per hidden layer, different hidden layers, different learning rates, different optimizers etc.

In Keras Tuner, `max_trials` controls the number of different model configurations (i.e., hyperparameter combinations) that will be tested during the hyperparameter search process. It does not necessarily test all possible combinations unless `max_trials` is set to the total number of combinations. 

Here's how it works:

1. **All Possible Combinations**: 
   - When you define hyperparameter ranges, Keras Tuner will generate all possible combinations based on the defined search space. For instance:
     - Neurons: `[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128]` (16 options)
     - Optimizers: `[opt1, opt2, opt3]` (3 options)
     - Learning rates: `[lr1, lr2, lr3, lr4]` (4 options)
   
   - Total combinations: `16 * 3 * 4 = 192`.

2. **max_trials**:
   - **max_trials = 192**: If you set `max_trials` equal to the total number of combinations (192 in this case), Keras Tuner will try all possible combinations.
   - **max_trials < 192**: If `max_trials` is less than 192, Keras Tuner will randomly sample `max_trials` combinations from the total search space. For instance, if `max_trials = 100`, it will randomly test 100 different combinations out of the 192 possible ones.
   - **max_trials > 192**: If `max_trials` is greater than the total number of combinations, Keras Tuner will only test each combination once (192 in this case) and ignore the extra trials.

In practice, you often use a smaller `max_trials` to save time and computational resources, especially if the search space is very large. Random sampling (when `max_trials` is less than the total combinations) can still yield good results because hyperparameter tuning often benefits from exploring diverse configurations rather than exhaustively searching all possible ones.

In [14]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
import keras_tuner as kt
import shutil
import os

In [23]:
df = pd.read_csv('datasets/HousingData.csv')
df_cleaned = df.dropna()
X = df_cleaned.drop(columns=['MEDV'])
y = df_cleaned['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)
mm  = MinMaxScaler()
mm.fit(X_train)
X_train = mm.transform(X_train)
X_test = mm.transform(X_test)

In [7]:
total_training_examples = X_train.shape[0]
total_training_examples

16374

In [10]:
total_features = X_train.shape[1]
total_features

112

In [19]:
def build_model(hp, input_shape, num_outputs):
    # total combinations for 2 choices of layers, 8 choices of neurons per layer and 3 choices of optimizers and 3 choices of lr: 2x8x3x3=144
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=input_shape))
    for i in range(hp.Int('num_layers', min_value=1, max_value=2)):
        model.add(tf.keras.layers.Dense(units=hp.Int('neuron'+str(i+1),
                                                     min_value=16,
                                                     max_value=128,
                                                     step=16),
                                                     activation='relu'))
    model.add(tf.keras.layers.Dense(1 if num_outputs == 2 else num_outputs))
    optimizer_choice = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('adam_learning_rate', [1e-3, 1e-2, 1e-1]))
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=hp.Choice('sgd_learning_rate', [1e-3, 1e-2, 1e-1]))
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(
            learning_rate=hp.Choice('rmsprop_learning_rate', [1e-3, 1e-2, 1e-1]))
    model.compile(optimizer=optimizer,
                loss='mse',
                metrics=['mean_absolute_percentage_error'])
    return model
        

In [22]:
tuner = kt.RandomSearch(
    hypermodel=lambda hp: build_model(hp, input_shape=(total_features, ), num_outputs=2),
    objective='val_mean_absolute_percentage_error',
    max_trials = 144,
    executions_per_trial=1,
    directory='.',
    project_name='my-tuner'
)

In [ ]:
full_path = os.path.join('.', 'my-tuner')

# Remove the directory if it exists
if os.path.exists(full_path):
    shutil.rmtree(full_path)
tuner.search(X_train, y_train, epochs=10, validation_split=0.2)